In [1]:
import MySQLdb
import sys
import bz2
import pandas as pd

In [2]:
# Virtual machine IP adress
HOST            = "148.60.11.195"

## MySQL user
DB_USER         = "script2"

## MySQL password
DB_PASSWD       = "ud6cw3xNRKnrOz6H"

In [3]:
# Virtual machine IP adress
HOST2            = "localhost"

## MySQL user
DB_USER2         = "root"

## MySQL password
DB_PASSWD2       = "password"

In [24]:
def get_sizes(cids):
    try:
        socket_new = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_result")
        socket_old = MySQLdb.connect(HOST2, DB_USER2, DB_PASSWD2, "IrmaDB_prod_backup")
        cursor_new = socket_new.cursor()
        cursor_old = socket_old.cursor()
        # retrieve the size of first cid
        query_new="SELECT compiled_kernel_size FROM `compilations` where cid={}".format(cids[1])
        query_old="SELECT core_size FROM `Compilations` where cid={}".format(cids[0])
        cursor_new.execute(query_new)
        cursor_old.execute(query_old)
        result_new=cursor_new.fetchone()
        #print("select compiled_kernel_size from compilations where cid={} : {}".format(cids[1], result1))
        result_old=cursor_old.fetchone()
        #print("select core_size from Compilations where cid={} : {}".format(cids[0], result2))
        return [result_old[0], result_new[0]]
    except Exception as e:
        print(str(e),"\n" + "Unable to connect to database", file=sys.stderr)
        exit(-1)
    finally:
        cursor_new.close()
        cursor_old.close()
        socket_new.close()
        socket_old.close()

In [38]:
l=[['100817', '74258'], ['101222', '74259'], ['101703', '74260'], ['101863', '74261'], \
   ['103074', '74262'], ['103430', '74263'], ['107624', '74264'], ['108678', '74265'], \
   ['109504', '74266'], ['109991', '74267'], ['110107', '74268'], ['110840', '74269'], \
   ['112869', '74270'], ['114194', '74271'], ['114560', '74272'], ['115083', '74273'], \
   ['116131', '74274'], ['30196', '74275'], ['30199', '74276'], ['30576', '74277'], \
   ['31167', '74278'], ['32131', '74279']]
tmp=[]
#pandas.DataFrame([{'A': 1.1, 'B': 2, 'C': 3.3, 'D': 4},
#    {'A': 2.7, 'B': 10, 'C': 5.4, 'D': 7},
#    {'A': 5.3, 'B': 9, 'C': 1.5, 'D': 15}])
for i in l:
    sizes=get_sizes(i)
    tmp.append({'cid_origin': i[0], 'size_origin': sizes[0], 'cid_mutated': i[1], 'size_mutated': sizes[1], \
                'difference': sizes[0]-sizes[1]})

res=pd.DataFrame(tmp)
print(res.columns)
print(res.index)
res

Index(['cid_mutated', 'cid_origin', 'difference', 'size_mutated',
       'size_origin'],
      dtype='object')
RangeIndex(start=0, stop=22, step=1)


,cid_mutated,cid_origin,difference,size_mutated,size_origin
0,74258,100817,2048384,30096112,32144496
1,74259,101222,2077184,27775800,29852984
2,74260,101703,2017392,12326632,14344024
3,74261,101863,2000896,23777408,25778304
4,74262,103074,1998984,21349952,23348936
5,74263,103430,2039464,16702648,18742112
6,74264,107624,-6032,16938856,16932824
7,74265,108678,1941016,28565760,30506776
8,74266,109504,1906832,33656944,35563776
9,74267,109991,1962736,31799096,33761832


In [35]:
try:
    socket_old = MySQLdb.connect(HOST2, DB_USER2, DB_PASSWD2, "IrmaDB_prod_backup")
    cursor_old = socket_old.cursor()
    query_old="SELECT config_file FROM `Compilations` where cid=107624"
    cursor_old.execute(query_old)
    result_old=cursor_old.fetchone()
    try:
        config_file_old = bz2.decompress(result_old[0]).decode('ascii')
    except Exception as e:
        print(str(e),"\n" + "Unable to decompress... ", file=sys.stderr)
        exit(-1)
        
    socket_new = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_result")
    cursor_new = socket_new.cursor()
    query_new="SELECT config_file FROM `compilations` where cid=74264"
    cursor_new.execute(query_new)
    result_new=cursor_new.fetchone()
    try:
        config_file_new = bz2.decompress(result_new[0]).decode('ascii')
    except Exception as e:
        print(str(e),"\n" + "Unable to decompress... ", file=sys.stderr)
        exit(-1)

    with open("config_old", 'w+') as c1:
        c1.write(config_file_old)
    with open("config_new", 'w+') as c2:
        c2.write(config_file_new)
    
except Exception as e:
    print(str(e),"\n" + "Unable to connect to database", file=sys.stderr)
    exit(-1)
finally:
    cursor_new.close()
    cursor_old.close()
    socket_new.close()
    socket_old.close()    

In [4]:
#-------------------- Get sets of identical configs and save result Dataframe ---------------------
try:
    socket = MySQLdb.connect(HOST, DB_USER, DB_PASSWD, "IrmaDB_result")
    cursor = socket.cursor()
    # firstly select the "basis" config files
    query="SELECT cid, config_file, compiled_kernel_size, compilation_time, hid FROM `compilations` where compiled_kernel_version='4.15' and (cid+200)%400=0 and cid>30000 and cid<72000"
    cursor.execute(query)
    result= cursor.fetchall()
    result=[list(i) for i in result]
    df = pd.DataFrame()
    # For each of the previously found cid (i.e our sample), we need to find the entries 
    # which have used the same .config file
    for row in result:
        query2="SELECT cid, compiled_kernel_size, compilation_time, hid FROM compilations \
                WHERE config_file = \
                    (SELECT config_file FROM compilations WHERE cid={})\
                    and cid!={} and cid>30000".format(row[0], row[0])
        cursor.execute(query2)
        matches=cursor.fetchall()
        # dumb conversion tuple -> list for format issues
        matches=[list(i) for i in matches]
        for i in matches:
            i.insert(0, row[0])
        labels=[i[0] for i in cursor.description]
        labels.insert(0, 'origin_config')
        temp=pd.DataFrame(matches, columns=labels)
        df=df.append(temp)
    df.to_pickle('DF2_kernel_gen')
except Exception as e:
    print(str(e),"\n" + "Unable to connect to database", file=sys.stderr)
    exit(-1)
finally:
    cursor.close()
    socket.close()
df

,origin_config,cid,compiled_kernel_size,compilation_time,hid
0,30200,74043,106632848,422.508,1881
1,30200,74044,106632848,423.235,1881
2,30200,74045,106632848,424.158,1881
3,30200,74046,106632848,422.291,1881
4,30200,74047,106632848,425.372,1881
0,30600,74048,231143736,260.940,1881
1,30600,74049,231145256,261.516,1881
2,30600,74050,231146176,262.388,1881
3,30600,74051,231142664,261.586,1881
4,30600,74052,231144528,262.498,1881
